In [1]:
import numpy as np
import json
import math
import time
import matplotlib.pyplot as plt
from util import matrix, cost, euclidean, checkCapacity as cc, ripupandrepair as rr
import random

In [2]:
paths = []
with open('data/paths.txt') as f:
    paths = f.read().split('\n')
print(paths)

['X-n101-k25', 'X-n115-k10', 'X-n261-k13', 'X-n308-k13', 'X-n429-k61', 'X-n524-k153', 'X-n819-k171', 'X-n856-k95', 'X-n936-k151', 'X-n1001-k43']


In [3]:
paths = ['X-n1001-k43']

In [4]:
types = ['greedy', 'radial', 'concentric', 'low-first']
types = ['low-first']

Using RnR (Rip up and Repair) techniques, nodes are swapped between strings

## find best string from shortened string and some points

In [ ]:
for path in paths:
    f = open('jsons/' + path + '.json')
    data = json.load(f)
    dim = data['dimension']
    demand = data['demand']
    depot = (data['depot'][0],data['depot'][1])
    x = data['x']
    y = data['y']
    print('\n----- ' + data['name'] + ' -----')
    for type in types:
        print('-- ' + type + ' --')

        # start chrono
        t1 = time.time()
        
        # calculate distance matrix
        distances = matrix.getDist(dim, x, y, euclidean.euc)

        strings = data[type]['initial']['solution'].copy()
        if type in data.keys():
            if 'optimized' not in data[type].keys():
                data[type]['optimized'] = {}
        else:
            print('no initial solution yet')

        # how many points to swap
        degree = 3
        
        # iterations
        n = 2
        k = 0
        
        # replaced n strings
        replaced = 0
        
        # start iterations
        while k < n:
            k+=1
            print('------------------------------------ iteration ' + str(k) + ' ------------------------------------')

            for j in range(len(strings)):
                for i in range(len(strings)):

                    # will exchange points between master and slave
                    master = strings[j]
                    slave = strings[i]
                    firstcostmaster = cost.cost(distances, master)
                    firstcostslave = cost.cost(distances, slave)
                    temporarymaster = master.copy()
                    temporaryslave = slave.copy()

                    d = degree
                    if degree > len(master)-2:
                        d = len(master) - 2
                    if degree > len(slave)-2:
                        d = len(slave)-2
                    shortest = len(slave)
                    if len(master) < len(slave):
                        shortest = len(master)
                    
                    if i==j:
                        continue

                    # take points from master
                    for u in range(d,shortest-1):
                        tempmaster = master.copy()
                        poolnodes = []
                        for q in range(d):
                            poolnodes.append(master[u-q])
                            tempmaster.pop(u-d+1)

                        # take points from slave
                        for v in range(d,shortest-1):
                            nodes = poolnodes.copy()
                            tempslave = slave.copy()
                            for q in range(d):
                                nodes.append(slave[v-q])
                                tempslave.pop(v-d+1)
                            
                            # swap points
                            newmaster, nmcost, remaining = rr.findOpt(tempmaster, master, nodes, distances, cost.cost, False)
                            if newmaster != master:
                                newslave, nscost, _ = rr.findOpt(tempslave, slave, remaining, distances, cost.cost, True)

                                # new cost better than master slave pair?
                                if nmcost + nscost < cost.cost(distances, master) + cost.cost(distances, slave):

                                    # new cost better than other swap pair?
                                    if nmcost + nscost < firstcostmaster + firstcostslave:

                                        # does new solution adhere to constraints?
                                        if cc.checkCapacity(data['capacity'],demand,newmaster) and cc.checkCapacity(data['capacity'],demand,newslave):
#                                             print('--- new cost: ' + str(nmcost + nscost) + ' (VS: ' + str(cost.cost(distances,master) + cost.cost(distances,slave)) + ')')
#                                             print('og master:  ' + str(master)    + ' \t cost: ' + str(cost.cost(distances, master)))
#                                             print('new master: ' + str(newmaster) + ' \t cost: ' + str(nmcost))
#                                             print('og slave:   ' + str(slave)     + ' \t cost: ' + str(cost.cost(distances, slave)))
#                                             print('new slave:  ' + str(newslave)  + ' \t cost: ' + str(nscost))
                                            strings[j] = newmaster
                                            strings[i] = newslave
                                            firstcostmaster = nmcost
                                            firstcostslave = nscost
                                            temporarymaster = newmaster
                                            temporaryslave = newslave
                                            replaced += 1

                # put master and slave back in place
                strings[j] = temporarymaster
                strings[i] = temporaryslave

        if not cc.checkTotal(strings, dim):
            print('ERROR: NOT ALL POINTS PRESENT')
        
        t2 = time.time()
        print('replaced ' + str(replaced) + ' strings in ' + str(round((t2-t1)*100)/100) + ' seconds')
        print('dropped cost from ' +
              str(np.sum([cost.cost(distances,string) for string in data[type]['initial']['solution']])) + 
              ' to ' +
              str(np.sum([cost.cost(distances,string) for string in strings]))
             )
            
        data[type]['optimized']['time'] = round((t2-t1)*1000)
        data[type]['optimized']['cost'] = int(np.sum([cost.cost(distances,string) for string in strings]))
        data[type]['optimized']['solution'] = strings
        
        with open('jsons/' + path + '.json', 'w') as outfile:
            json.dump(data, outfile)


----- X-n1001-k43 -----
-- low-first --
------------------------------------ iteration 1 ------------------------------------
